In [1]:
# 1.Load the dataset & show summary

import numpy as np
import pandas as pd

patientsdata = pd.read_csv("patient_Data.csv")
billingdata = pd.read_csv("billing_Data.csv")

print("\n---- PATIENTS ----")
print(patientsdata.info())
print(patientsdata.head())

print("\n---- BILLING ----")
print(billingdata.info())
print(billingdata.head())


---- PATIENTS ----
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   PatientID       6 non-null      int64  
 1   Name            6 non-null      object 
 2   Department      6 non-null      object 
 3   Doctor          6 non-null      object 
 4   BillAmount      4 non-null      float64
 5   ReceptionistID  6 non-null      int64  
 6   CheckInTime     6 non-null      object 
dtypes: float64(1), int64(2), object(4)
memory usage: 468.0+ bytes
None
   PatientID     Name   Department     Doctor  BillAmount  ReceptionistID  \
0        101    Alice   Cardiology  Dr. Smith      5000.0               1   
1        102      Bob    Neurology   Dr. John         NaN               2   
2        103  Charlie  Orthopedics    Dr. Lee      7500.0               1   
3        104    David   Cardiology  Dr. Smith      6200.0               3   
4        105      

In [2]:
# 2.Select only columns relevant for billing

billing = patientsdata[['PatientID', 'Department', 'Doctor', 'BillAmount']]
print("---- BILLING COLUMNS ----")
print(billing.head())


---- BILLING COLUMNS ----
   PatientID   Department     Doctor  BillAmount
0        101   Cardiology  Dr. Smith      5000.0
1        102    Neurology   Dr. John         NaN
2        103  Orthopedics    Dr. Lee      7500.0
3        104   Cardiology  Dr. Smith      6200.0
4        105  Dermatology   Dr. Rose         NaN


In [3]:
# 3. Drop administrative columns

print("---- DROPPING ADMIN COLUMNS ----")
patientsdata.drop(columns=['ReceptionistID', 'CheckInTime'],inplace=True)
patientsdata

---- DROPPING ADMIN COLUMNS ----


,PatientID,Name,Department,Doctor,BillAmount
0,101,Alice,Cardiology,Dr. Smith,5000.0
1,102,Bob,Neurology,Dr. John,NaN
2,103,Charlie,Orthopedics,Dr. Lee,7500.0
3,104,David,Cardiology,Dr. Smith,6200.0
4,105,Eva,Dermatology,Dr. Rose,NaN
5,101,Alice,Cardiology,Dr. Smith,5000.0


In [4]:
# 4. Total bill amount per department

print("---- TOTAL BILL AMOUNT PER DEPARTMENT ----")
print(billing.groupby('Department')['BillAmount'].sum())

---- TOTAL BILL AMOUNT PER DEPARTMENT ----
Department
Cardiology     16200.0
Dermatology        0.0
Neurology          0.0
Orthopedics     7500.0
Name: BillAmount, dtype: float64


In [5]:
# 5. Remove duplicate patient records

print("---- AFTER REMOVING DUPLICATES ----")
updated=billing.drop_duplicates("PatientID")
updated

---- AFTER REMOVING DUPLICATES ----


,PatientID,Department,Doctor,BillAmount
0,101,Cardiology,Dr. Smith,5000.0
1,102,Neurology,Dr. John,NaN
2,103,Orthopedics,Dr. Lee,7500.0
3,104,Cardiology,Dr. Smith,6200.0
4,105,Dermatology,Dr. Rose,NaN


In [6]:
# 6. Fill missing BillAmount with mean

print("---- FILLING MISSING VALUES WITH MEAN ----")
updated["BillAmount"].fillna(updated["BillAmount"].mean())


---- FILLING MISSING VALUES WITH MEAN ----


0    5000.000000
1    6233.333333
2    7500.000000
3    6200.000000
4    6233.333333
Name: BillAmount, dtype: float64

In [7]:
billingdata

,PatientID,InsuranceCovered,FinalAmount
0,101,2000,3000
1,102,1500,3500
2,103,2500,5000
3,104,3000,3200
4,105,1000,4000


In [8]:
# 7. Merge patient & billing datasets

print("---- MERGE DATASET ----")
merge_ds=pd.merge(updated,billingdata,on="PatientID")
merge_ds

---- MERGE DATASET ----


,PatientID,Department,Doctor,BillAmount,InsuranceCovered,FinalAmount
0,101,Cardiology,Dr. Smith,5000.0,2000,3000
1,102,Neurology,Dr. John,NaN,1500,3500
2,103,Orthopedics,Dr. Lee,7500.0,2500,5000
3,104,Cardiology,Dr. Smith,6200.0,3000,3200
4,105,Dermatology,Dr. Rose,NaN,1000,4000


In [9]:
# 8. Concatenate new patients (row-wise)

print("---- AFTER ADDING NEW PATIENTS ----")
dfr=pd.DataFrame({
    "PatientID": [106,107],
    "Department": ["Cardiology","Neurology"],
    "Doctor": ["Dr. Iybun","Dr. Iylu"],
    "BillAmount": [5500.0,8300.0],
    #"InsuranceCovered": [2000,3000],
    #"FinalAmount": [3500,5300]
})
updatedrows=pd.concat([merge_ds,dfr],ignore_index=True)
updatedrows

---- AFTER ADDING NEW PATIENTS ----


,PatientID,Department,Doctor,BillAmount,InsuranceCovered,FinalAmount
0,101,Cardiology,Dr. Smith,5000.0,2000.0,3000.0
1,102,Neurology,Dr. John,NaN,1500.0,3500.0
2,103,Orthopedics,Dr. Lee,7500.0,2500.0,5000.0
3,104,Cardiology,Dr. Smith,6200.0,3000.0,3200.0
4,105,Dermatology,Dr. Rose,NaN,1000.0,4000.0
5,106,Cardiology,Dr. Iybun,5500.0,NaN,NaN
6,107,Neurology,Dr. Iylu,8300.0,NaN,NaN


In [10]:
# 9. Concatenate new patients (col-wise)

print("---- FINAL DATASET ----")
dfc=pd.DataFrame({
    "InsuranceCovered": [2000,3000],
    "FinalAmount": [3500,5300]
})

updatedc=pd.concat([updatedrows,dfc],axis=1,ignore_index=True)
updatedc

---- FINAL DATASET ----


,0,1,2,3,4,5,6,7
0,101,Cardiology,Dr. Smith,5000.0,2000.0,3000.0,2000.0,3500.0
1,102,Neurology,Dr. John,NaN,1500.0,3500.0,3000.0,5300.0
2,103,Orthopedics,Dr. Lee,7500.0,2500.0,5000.0,NaN,NaN
3,104,Cardiology,Dr. Smith,6200.0,3000.0,3200.0,NaN,NaN
4,105,Dermatology,Dr. Rose,NaN,1000.0,4000.0,NaN,NaN
5,106,Cardiology,Dr. Iybun,5500.0,NaN,NaN,NaN,NaN
6,107,Neurology,Dr. Iylu,8300.0,NaN,NaN,NaN,NaN


In [11]:
# the final one

print("---- FINAL CLEANED DS ----")
df=pd.DataFrame({
    "PatientID": [106,107],
    "Department": ["Cardiology","Neurology"],
    "Doctor": ["Dr. Iybun","Dr. Iylu"],
    "BillAmount": [5500.0,8300.0],
    "InsuranceCovered": [2000,3000],
    "FinalAmount": [3500,5300]
})
thefinal=pd.concat([merge_ds,df],ignore_index=True)
thefinal

---- FINAL CLEANED DS ----


,PatientID,Department,Doctor,BillAmount,InsuranceCovered,FinalAmount
0,101,Cardiology,Dr. Smith,5000.0,2000,3000
1,102,Neurology,Dr. John,NaN,1500,3500
2,103,Orthopedics,Dr. Lee,7500.0,2500,5000
3,104,Cardiology,Dr. Smith,6200.0,3000,3200
4,105,Dermatology,Dr. Rose,NaN,1000,4000
5,106,Cardiology,Dr. Iybun,5500.0,2000,3500
6,107,Neurology,Dr. Iylu,8300.0,3000,5300
